In [1]:
# predict the next char
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import re
%matplotlib inline

In [2]:
def make_mini_batch(train_data, size_of_mini_batch, length_of_sequences):
    inputs  = ''
    outputs = ''
    for _ in range(size_of_mini_batch):
        index   = random.randint(0, len(train_data) - length_of_sequences - 1)
        inputs  += train_data[index:index + length_of_sequences]
        outputs += train_data[index + length_of_sequences]
    inputs = np.eye(num_of_char)[[ord(x) for x in inputs if ord(x) < 128]]
    outputs = np.eye(num_of_char)[[ord(x) for x in outputs if ord(x) < 128]]
    inputs  = inputs.reshape(-1, length_of_sequences, 128)
    outputs = outputs.reshape(-1, 128)
    return (inputs, outputs)

In [3]:
def make_prediction_initial(train_data, index, length_of_sequences):
    inputs = train_data[index:index + length_of_sequences]
    return np.eye(num_of_char)[[ord(x) for x in inputs if ord(x) < 128]]    
#inputs, _  = make_prediction_initial(train_data, 0, length_of_initial_sequences)
#inputs.shape

In [25]:
train_data_path             = "./input/linux.c"
num_of_char                 = 128
num_of_input_nodes          = 128
num_of_hidden_nodes         = 512
num_of_output_nodes         = 128
length_of_sequences         = 50
num_of_training_epochs      = 25100
length_of_initial_sequences = 50
num_of_prediction_epochs    = 1000
size_of_mini_batch          = 20
learning_rate               = 0.0002
forget_bias                 = 1.0
train_dir                   = 'train/'

In [5]:
#train_data = np.load(train_data_path)
f = open(train_data_path)
text = f.read()
f.close()
train_data = ''.join([x for x in text if ord(x) < 128])

In [6]:
def inference(input_ph, size_of_mini_batch_ph, lstm_output_keep_prob):
    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var   = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var   = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        weight1_hist = tf.histogram_summary("layer1/weights", weight1_var)
        weight2_hist = tf.histogram_summary("layer2/weights", weight2_var)
        bias1_hist = tf.histogram_summary("layer1/biases", bias1_var)
        bias2_hist = tf.histogram_summary("layer2/biases", bias2_var)
        
        # pre rnn
        in1 = tf.transpose(input_ph, [1, 0, 2])         # (batch, sequence, data) -> (sequence, batch, data)
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) # (sequence, batch, data) -> (sequence * batch, data)
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, length_of_sequences, in3)     # sequence * (batch, data)

        cell = tf.nn.rnn_cell.BasicLSTMCell(num_of_hidden_nodes, forget_bias=forget_bias, state_is_tuple=True)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=lstm_output_keep_prob)
        #cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2, state_is_tuple=True)
        initial_state = cell.zero_state(size_of_mini_batch_ph, tf.float32)
        rnn_output, states_op = tf.nn.rnn(cell, in4, initial_state=initial_state, dtype=tf.float32)

        output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
    return output_op, rnn_output

In [7]:
def loss(output_op):
    with tf.name_scope("loss") as scope:
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_op, supervisor_ph))
        loss_op = cross_entropy
        tf.scalar_summary("loss", loss_op)
    return loss_op

In [8]:
def train(loss_op):
    with tf.name_scope("training") as scope:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        #optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss_op)
    return training_op

In [26]:
with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    size_of_mini_batch_ph = tf.placeholder(tf.int32, name="size_of_mini_batch")
    lstm_output_keep_prob_ph = tf.placeholder(tf.float32, name="lstm_output_keep_prob")

    output_op, rnn_output = inference(input_ph, size_of_mini_batch_ph, lstm_output_keep_prob_ph)
    loss_op = loss(output_op)
    training_op = train(loss_op)

    summary_op = tf.merge_all_summaries()

    with tf.Session() as sess:
        summary_writer = tf.train.SummaryWriter("data", graph=sess.graph)
        """
        # random seed fix
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        """
        epoch_start = 0
        ckpt = tf.train.get_checkpoint_state(train_dir)
        saver = tf.train.Saver()
        if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
            print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            epoch_start = int(re.search(r"(\d+)$", ckpt.model_checkpoint_path).group())
        else:
            print("Created model with fresh parameters.")
            sess.run(tf.initialize_all_variables())

        for epoch in range(epoch_start, num_of_training_epochs):
            inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)

            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                size_of_mini_batch_ph: size_of_mini_batch,
                lstm_output_keep_prob_ph: 0.5,
            }
            rnnout, _ = sess.run([rnn_output, training_op], feed_dict=train_dict)

            if epoch % 10 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                summary_writer.add_summary(summary_str, epoch)
                print("train#%d, train loss: %e" % (epoch, train_loss))

            if epoch % 100 == 0:
                saver.save(sess, train_dir + "translate.ckpt", global_step=epoch)
                print("checkpoint saved %d" % epoch)

#        inputs = make_prediction_initial(train_data, len(train_data)-length_of_sequences, length_of_initial_sequences)
        
        for t in [300, 600, 900, 1200, 1300]:
            inputs = make_prediction_initial(train_data, len(train_data) - t, length_of_initial_sequences)
            outputs = np.array([[0 for _ in range(128)]])

            print("prediction: ")
            for epoch in range(num_of_prediction_epochs):
                pred_dict = {
                    input_ph:  [inputs],
                    size_of_mini_batch_ph: 1,
                    lstm_output_keep_prob_ph: 1.0
                }
                output = sess.run(output_op, feed_dict=pred_dict)
                output_onehotvec = np.eye(num_of_char)[[np.argmax(output)]]
                inputs  = np.delete(inputs, 0, 0)
                inputs  = np.append(inputs, output_onehotvec, 0)
                outputs = np.append(outputs, output_onehotvec, 0)

            outputs  = np.delete(outputs, 0, 0)
            output_ascii = np.argmax(outputs.reshape(num_of_prediction_epochs, num_of_output_nodes), axis=1)
            print("outputs:", ''.join([chr(x) for x in output_ascii]))

Reading model parameters from train/translate.ckpt-25100
prediction: 
('outputs:', ' thread_state_cpu_state(struct trace_state *state)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n{\n\tstruct trace_state_state(struct task_struct *p)\n

In [ ]:
"""
Reading model parameters from train/translate.ckpt-20300
train#20300, train loss: 2.308057e+00
checkpoint saved 20300
train#20310, train loss: 1.536508e+00
train#20320, train loss: 1.733895e+00
train#20330, train loss: 1.606260e+00
train#20340, train loss: 1.499637e+00
train#20350, train loss: 1.565281e+00
train#20360, train loss: 1.762253e+00
train#20370, train loss: 8.873143e-01
train#20380, train loss: 8.700088e-01
train#20390, train loss: 1.652048e+00
train#20400, train loss: 1.269021e+00
checkpoint saved 20400
train#20410, train loss: 1.913951e+00
train#20420, train loss: 1.209362e+00
train#20430, train loss: 1.958714e+00
train#20440, train loss: 1.320982e+00
train#20450, train loss: 1.293011e+00
train#20460, train loss: 1.869075e+00
train#20470, train loss: 1.828329e+00
train#20480, train loss: 1.618702e+00
train#20490, train loss: 2.237145e+00
train#20500, train loss: 1.597197e+00
checkpoint saved 20500
train#20510, train loss: 1.507922e+00
train#20520, train loss: 1.487552e+00
train#20530, train loss: 1.512341e+00
train#20540, train loss: 2.038653e+00
train#20550, train loss: 1.587827e+00
train#20560, train loss: 1.593687e+00
train#20570, train loss: 1.313300e+00
train#20580, train loss: 2.262438e+00
train#20590, train loss: 1.773277e+00
train#20600, train loss: 1.980995e+00
checkpoint saved 20600
train#20610, train loss: 2.282307e+00
train#20620, train loss: 1.634784e+00
train#20630, train loss: 1.133318e+00
train#20640, train loss: 1.899055e+00
train#20650, train loss: 1.657991e+00
train#20660, train loss: 1.581671e+00
train#20670, train loss: 1.528255e+00
train#20680, train loss: 1.479621e+00
train#20690, train loss: 1.369011e+00
train#20700, train loss: 1.223903e+00
checkpoint saved 20700
train#20710, train loss: 1.900427e+00
train#20720, train loss: 1.504781e+00
train#20730, train loss: 1.339800e+00
train#20740, train loss: 9.398098e-01
train#20750, train loss: 2.033169e+00
train#20760, train loss: 1.158042e+00
train#20770, train loss: 1.543935e+00
train#20780, train loss: 1.554647e+00
train#20790, train loss: 1.176221e+00
train#20800, train loss: 1.902539e+00
checkpoint saved 20800
train#20810, train loss: 1.283423e+00
train#20820, train loss: 1.636620e+00
train#20830, train loss: 1.642832e+00
train#20840, train loss: 1.269842e+00
train#20850, train loss: 1.930488e+00
train#20860, train loss: 1.715862e+00
train#20870, train loss: 2.394320e+00
train#20880, train loss: 1.060742e+00
train#20890, train loss: 1.127545e+00
train#20900, train loss: 1.060988e+00
checkpoint saved 20900
train#20910, train loss: 2.136998e+00
train#20920, train loss: 1.435570e+00
train#20930, train loss: 1.824119e+00
train#20940, train loss: 1.830916e+00
train#20950, train loss: 1.538578e+00
train#20960, train loss: 1.205606e+00
train#20970, train loss: 1.402751e+00
train#20980, train loss: 2.373272e+00
train#20990, train loss: 1.268400e+00
train#21000, train loss: 1.328629e+00
checkpoint saved 21000
train#21010, train loss: 1.388360e+00
train#21020, train loss: 1.220459e+00
train#21030, train loss: 1.367111e+00
train#21040, train loss: 1.858546e+00
train#21050, train loss: 1.591820e+00
train#21060, train loss: 2.741305e+00
train#21070, train loss: 1.442837e+00
train#21080, train loss: 1.728130e+00
train#21090, train loss: 1.790982e+00
train#21100, train loss: 1.430306e+00
checkpoint saved 21100
train#21110, train loss: 1.622952e+00
train#21120, train loss: 2.445496e+00
train#21130, train loss: 1.355362e+00
train#21140, train loss: 1.717934e+00
train#21150, train loss: 1.205231e+00
train#21160, train loss: 1.958937e+00
train#21170, train loss: 1.654479e+00
train#21180, train loss: 1.894882e+00
train#21190, train loss: 1.497668e+00
train#21200, train loss: 1.464053e+00
checkpoint saved 21200
train#21210, train loss: 1.316254e+00
train#21220, train loss: 1.404847e+00
train#21230, train loss: 2.146140e+00
train#21240, train loss: 1.804934e+00
train#21250, train loss: 1.465032e+00
train#21260, train loss: 2.069741e+00
train#21270, train loss: 2.399488e+00
train#21280, train loss: 1.761610e+00
train#21290, train loss: 1.738571e+00
train#21300, train loss: 1.473338e+00
checkpoint saved 21300
train#21310, train loss: 1.870548e+00
train#21320, train loss: 2.096689e+00
train#21330, train loss: 1.560470e+00
train#21340, train loss: 1.426763e+00
train#21350, train loss: 1.499621e+00
train#21360, train loss: 2.033963e+00
train#21370, train loss: 1.623849e+00
train#21380, train loss: 1.467345e+00
train#21390, train loss: 2.716572e+00
train#21400, train loss: 2.052765e+00
checkpoint saved 21400
train#21410, train loss: 1.519114e+00
train#21420, train loss: 1.581319e+00
train#21430, train loss: 1.271940e+00
train#21440, train loss: 1.680888e+00
train#21450, train loss: 1.692224e+00
train#21460, train loss: 2.085391e+00
train#21470, train loss: 1.550083e+00
train#21480, train loss: 1.624735e+00
train#21490, train loss: 1.498488e+00
train#21500, train loss: 2.593135e+00
checkpoint saved 21500
train#21510, train loss: 1.193682e+00
train#21520, train loss: 1.984551e+00
train#21530, train loss: 1.340804e+00
train#21540, train loss: 1.947179e+00
train#21550, train loss: 1.239280e+00
train#21560, train loss: 1.729396e+00
train#21570, train loss: 1.641453e+00
train#21580, train loss: 1.340688e+00
train#21590, train loss: 1.020995e+00
train#21600, train loss: 1.672262e+00
checkpoint saved 21600
train#21610, train loss: 1.938816e+00
train#21620, train loss: 9.738945e-01
train#21630, train loss: 9.570931e-01
train#21640, train loss: 1.563127e+00
train#21650, train loss: 1.924654e+00
train#21660, train loss: 1.898378e+00
train#21670, train loss: 2.247866e+00
train#21680, train loss: 1.386500e+00
train#21690, train loss: 2.233772e+00
train#21700, train loss: 2.142974e+00
checkpoint saved 21700
train#21710, train loss: 1.766569e+00
train#21720, train loss: 1.809919e+00
train#21730, train loss: 1.063094e+00
train#21740, train loss: 1.356151e+00
train#21750, train loss: 1.778495e+00
train#21760, train loss: 3.081476e+00
train#21770, train loss: 1.293678e+00
train#21780, train loss: 1.477529e+00
train#21790, train loss: 1.428364e+00
train#21800, train loss: 1.834625e+00
checkpoint saved 21800
train#21810, train loss: 1.748480e+00
train#21820, train loss: 1.674332e+00
train#21830, train loss: 1.065654e+00
train#21840, train loss: 1.198339e+00
train#21850, train loss: 1.750440e+00
train#21860, train loss: 1.878578e+00
train#21870, train loss: 1.814905e+00
train#21880, train loss: 1.659134e+00
train#21890, train loss: 1.616775e+00
train#21900, train loss: 1.713840e+00
checkpoint saved 21900
train#21910, train loss: 1.786170e+00
train#21920, train loss: 1.812722e+00
train#21930, train loss: 2.639908e+00
train#21940, train loss: 1.204934e+00
train#21950, train loss: 1.894368e+00
train#21960, train loss: 1.295553e+00
train#21970, train loss: 1.447294e+00
train#21980, train loss: 1.754908e+00
train#21990, train loss: 8.426272e-01
train#22000, train loss: 1.543515e+00
checkpoint saved 22000
train#22010, train loss: 1.299196e+00
train#22020, train loss: 1.631231e+00
train#22030, train loss: 8.199859e-01
train#22040, train loss: 1.916002e+00
train#22050, train loss: 1.525439e+00
train#22060, train loss: 1.342061e+00
train#22070, train loss: 1.295705e+00
train#22080, train loss: 1.791689e+00
train#22090, train loss: 9.095632e-01
train#22100, train loss: 1.567351e+00
checkpoint saved 22100
train#22110, train loss: 1.972336e+00
train#22120, train loss: 1.074589e+00
train#22130, train loss: 1.697042e+00
train#22140, train loss: 1.718283e+00
train#22150, train loss: 9.359987e-01
train#22160, train loss: 1.304575e+00
train#22170, train loss: 1.361005e+00
train#22180, train loss: 1.015699e+00
train#22190, train loss: 1.745779e+00
train#22200, train loss: 1.433720e+00
checkpoint saved 22200
train#22210, train loss: 9.295467e-01
train#22220, train loss: 1.411549e+00
train#22230, train loss: 1.150725e+00
train#22240, train loss: 2.048058e+00
train#22250, train loss: 1.436642e+00
train#22260, train loss: 1.505169e+00
train#22270, train loss: 7.607541e-01
train#22280, train loss: 1.892846e+00
train#22290, train loss: 2.062000e+00
train#22300, train loss: 1.330914e+00
checkpoint saved 22300
train#22310, train loss: 1.259406e+00
train#22320, train loss: 1.686425e+00
train#22330, train loss: 1.004243e+00
train#22340, train loss: 1.268478e+00
train#22350, train loss: 1.189848e+00
train#22360, train loss: 9.955730e-01
train#22370, train loss: 1.651509e+00
train#22380, train loss: 1.120237e+00
train#22390, train loss: 1.723662e+00
train#22400, train loss: 1.194300e+00
checkpoint saved 22400
train#22410, train loss: 2.141006e+00
train#22420, train loss: 1.561045e+00
train#22430, train loss: 1.425071e+00
train#22440, train loss: 1.611368e+00
train#22450, train loss: 1.106680e+00
train#22460, train loss: 1.283139e+00
train#22470, train loss: 1.664788e+00
train#22480, train loss: 1.733953e+00
train#22490, train loss: 1.401684e+00
train#22500, train loss: 1.491285e+00
checkpoint saved 22500
train#22510, train loss: 2.015242e+00
train#22520, train loss: 1.482186e+00
train#22530, train loss: 1.415194e+00
train#22540, train loss: 1.805087e+00
train#22550, train loss: 1.119591e+00
train#22560, train loss: 1.461408e+00
train#22570, train loss: 9.233665e-01
train#22580, train loss: 2.559001e+00
train#22590, train loss: 1.106476e+00
train#22600, train loss: 1.567506e+00
checkpoint saved 22600
train#22610, train loss: 1.797762e+00
train#22620, train loss: 9.090668e-01
train#22630, train loss: 1.550624e+00
train#22640, train loss: 1.911655e+00
train#22650, train loss: 1.740026e+00
train#22660, train loss: 1.399742e+00
train#22670, train loss: 1.348730e+00
train#22680, train loss: 1.850097e+00
train#22690, train loss: 2.008671e+00
train#22700, train loss: 1.248700e+00
checkpoint saved 22700
train#22710, train loss: 1.631814e+00
train#22720, train loss: 1.914256e+00
train#22730, train loss: 1.687478e+00
train#22740, train loss: 1.187821e+00
train#22750, train loss: 1.359270e+00
train#22760, train loss: 1.556414e+00
train#22770, train loss: 1.464401e+00
train#22780, train loss: 1.718230e+00
train#22790, train loss: 2.326944e+00
train#22800, train loss: 1.813346e+00
checkpoint saved 22800
train#22810, train loss: 2.088521e+00
train#22820, train loss: 1.559488e+00
train#22830, train loss: 9.207589e-01
train#22840, train loss: 1.260037e+00
train#22850, train loss: 2.198026e+00
train#22860, train loss: 1.646446e+00
train#22870, train loss: 1.400100e+00
train#22880, train loss: 1.482791e+00
train#22890, train loss: 1.712660e+00
train#22900, train loss: 1.238551e+00
checkpoint saved 22900
train#22910, train loss: 1.616328e+00
train#22920, train loss: 1.177425e+00
train#22930, train loss: 1.103254e+00
train#22940, train loss: 1.584010e+00
train#22950, train loss: 1.126483e+00
train#22960, train loss: 1.335527e+00
train#22970, train loss: 1.763158e+00
train#22980, train loss: 1.832324e+00
train#22990, train loss: 1.845798e+00
train#23000, train loss: 2.208399e+00
checkpoint saved 23000
train#23010, train loss: 1.107256e+00
train#23020, train loss: 1.063667e+00
train#23030, train loss: 1.108641e+00
train#23040, train loss: 1.409725e+00
train#23050, train loss: 1.590454e+00
train#23060, train loss: 1.890473e+00
train#23070, train loss: 9.580496e-01
train#23080, train loss: 1.540698e+00
train#23090, train loss: 1.906145e+00
train#23100, train loss: 1.055443e+00
checkpoint saved 23100
train#23110, train loss: 1.712048e+00
train#23120, train loss: 1.788504e+00
train#23130, train loss: 1.628612e+00
train#23140, train loss: 2.325188e+00
train#23150, train loss: 1.243170e+00
train#23160, train loss: 1.615187e+00
train#23170, train loss: 7.087324e-01
train#23180, train loss: 1.783110e+00
train#23190, train loss: 8.466765e-01
train#23200, train loss: 1.912992e+00
checkpoint saved 23200
train#23210, train loss: 1.541012e+00
train#23220, train loss: 1.609977e+00
train#23230, train loss: 8.552257e-01
train#23240, train loss: 1.266610e+00
train#23250, train loss: 1.626974e+00
train#23260, train loss: 1.522677e+00
train#23270, train loss: 1.510802e+00
train#23280, train loss: 1.932441e+00
train#23290, train loss: 1.583061e+00
train#23300, train loss: 1.625309e+00
checkpoint saved 23300
train#23310, train loss: 1.941622e+00
train#23320, train loss: 1.580673e+00
train#23330, train loss: 1.419805e+00
train#23340, train loss: 2.069921e+00
train#23350, train loss: 1.578705e+00
train#23360, train loss: 1.548131e+00
train#23370, train loss: 1.736713e+00
train#23380, train loss: 1.361568e+00
train#23390, train loss: 1.370048e+00
train#23400, train loss: 1.668750e+00
checkpoint saved 23400
train#23410, train loss: 1.876574e+00
train#23420, train loss: 9.170253e-01
train#23430, train loss: 1.779171e+00
train#23440, train loss: 1.496177e+00
train#23450, train loss: 1.920108e+00
train#23460, train loss: 1.977345e+00
train#23470, train loss: 9.405708e-01
train#23480, train loss: 1.624682e+00
train#23490, train loss: 2.523500e+00
train#23500, train loss: 1.602145e+00
checkpoint saved 23500
train#23510, train loss: 1.529682e+00
train#23520, train loss: 1.239388e+00
train#23530, train loss: 1.535188e+00
train#23540, train loss: 1.941250e+00
train#23550, train loss: 1.730087e+00
train#23560, train loss: 1.341717e+00
train#23570, train loss: 1.260996e+00
train#23580, train loss: 1.124714e+00
train#23590, train loss: 1.471656e+00
train#23600, train loss: 1.053097e+00
checkpoint saved 23600
train#23610, train loss: 1.377022e+00
train#23620, train loss: 1.927820e+00
train#23630, train loss: 1.279255e+00
train#23640, train loss: 1.362103e+00
train#23650, train loss: 1.200208e+00
train#23660, train loss: 1.561851e+00
train#23670, train loss: 2.304060e+00
train#23680, train loss: 1.996603e+00
train#23690, train loss: 2.195233e+00
train#23700, train loss: 1.103516e+00
checkpoint saved 23700
train#23710, train loss: 1.710408e+00
train#23720, train loss: 1.328513e+00
train#23730, train loss: 1.088721e+00
train#23740, train loss: 1.347178e+00
train#23750, train loss: 1.641563e+00
train#23760, train loss: 2.049882e+00
train#23770, train loss: 1.998766e+00
train#23780, train loss: 1.760758e+00
train#23790, train loss: 1.419205e+00
train#23800, train loss: 1.354506e+00
checkpoint saved 23800
train#23810, train loss: 1.546582e+00
train#23820, train loss: 1.801242e+00
train#23830, train loss: 2.012899e+00
train#23840, train loss: 1.486718e+00
train#23850, train loss: 2.074677e+00
train#23860, train loss: 2.088813e+00
train#23870, train loss: 1.075516e+00
train#23880, train loss: 1.652897e+00
train#23890, train loss: 1.328241e+00
train#23900, train loss: 1.840466e+00
checkpoint saved 23900
train#23910, train loss: 1.717930e+00
train#23920, train loss: 1.515084e+00
train#23930, train loss: 1.717657e+00
train#23940, train loss: 2.579374e+00
train#23950, train loss: 1.657622e+00
train#23960, train loss: 1.236773e+00
train#23970, train loss: 2.256894e+00
train#23980, train loss: 1.507868e+00
train#23990, train loss: 1.420333e+00
train#24000, train loss: 1.574338e+00
checkpoint saved 24000
train#24010, train loss: 1.085680e+00
train#24020, train loss: 1.115443e+00
train#24030, train loss: 1.742365e+00
train#24040, train loss: 1.821784e+00
train#24050, train loss: 1.628793e+00
train#24060, train loss: 1.542848e+00
train#24070, train loss: 1.303705e+00
train#24080, train loss: 1.323099e+00
train#24090, train loss: 9.216894e-01
train#24100, train loss: 1.472391e+00
checkpoint saved 24100
train#24110, train loss: 1.217037e+00
train#24120, train loss: 2.156757e+00
train#24130, train loss: 1.849188e+00
train#24140, train loss: 1.976115e+00
train#24150, train loss: 1.966829e+00
train#24160, train loss: 1.330369e+00
train#24170, train loss: 1.696535e+00
train#24180, train loss: 1.240753e+00
train#24190, train loss: 1.176231e+00
train#24200, train loss: 1.614034e+00
checkpoint saved 24200
train#24210, train loss: 1.642643e+00
train#24220, train loss: 1.317912e+00
train#24230, train loss: 1.882313e+00
train#24240, train loss: 1.555265e+00
train#24250, train loss: 1.677539e+00
train#24260, train loss: 1.289503e+00
train#24270, train loss: 1.318498e+00
train#24280, train loss: 1.630337e+00
train#24290, train loss: 6.974375e-01
train#24300, train loss: 1.387676e+00
checkpoint saved 24300
train#24310, train loss: 7.233995e-01
train#24320, train loss: 1.418673e+00
train#24330, train loss: 1.355594e+00
train#24340, train loss: 1.341201e+00
train#24350, train loss: 9.942357e-01
train#24360, train loss: 1.339745e+00
train#24370, train loss: 1.289983e+00
train#24380, train loss: 1.950807e+00
train#24390, train loss: 1.478575e+00
train#24400, train loss: 1.499806e+00
checkpoint saved 24400
train#24410, train loss: 1.429473e+00
train#24420, train loss: 1.965558e+00
train#24430, train loss: 1.104218e+00
train#24440, train loss: 2.106326e+00
train#24450, train loss: 1.521921e+00
train#24460, train loss: 1.602053e+00
train#24470, train loss: 2.089900e+00
train#24480, train loss: 1.917457e+00
train#24490, train loss: 1.403337e+00
train#24500, train loss: 1.481863e+00
checkpoint saved 24500
train#24510, train loss: 1.355008e+00
train#24520, train loss: 1.214366e+00
train#24530, train loss: 1.493857e+00
train#24540, train loss: 1.359102e+00
train#24550, train loss: 1.432974e+00
train#24560, train loss: 1.164027e+00
train#24570, train loss: 7.519774e-01
train#24580, train loss: 2.182181e+00
train#24590, train loss: 1.358496e+00
train#24600, train loss: 1.448078e+00
checkpoint saved 24600
train#24610, train loss: 1.444953e+00
train#24620, train loss: 1.853512e+00
train#24630, train loss: 1.115489e+00
train#24640, train loss: 1.554228e+00
train#24650, train loss: 1.556967e+00
train#24660, train loss: 1.158468e+00
train#24670, train loss: 1.525058e+00
train#24680, train loss: 1.907090e+00
train#24690, train loss: 7.587268e-01
train#24700, train loss: 1.646027e+00
checkpoint saved 24700
train#24710, train loss: 1.056538e+00
train#24720, train loss: 1.904283e+00
train#24730, train loss: 1.483790e+00
train#24740, train loss: 1.669266e+00
train#24750, train loss: 1.855109e+00
train#24760, train loss: 1.734922e+00
train#24770, train loss: 1.517792e+00
train#24780, train loss: 1.832828e+00
train#24790, train loss: 1.961267e+00
train#24800, train loss: 2.272448e+00
checkpoint saved 24800
train#24810, train loss: 1.608313e+00
train#24820, train loss: 1.653549e+00
train#24830, train loss: 1.830196e+00
train#24840, train loss: 2.322645e+00
train#24850, train loss: 1.544685e+00
train#24860, train loss: 1.946487e+00
train#24870, train loss: 1.269829e+00
train#24880, train loss: 1.389449e+00
train#24890, train loss: 2.266294e+00
train#24900, train loss: 7.939113e-01
checkpoint saved 24900
train#24910, train loss: 2.424143e+00
train#24920, train loss: 2.116699e+00
train#24930, train loss: 8.167627e-01
train#24940, train loss: 2.201729e+00
train#24950, train loss: 1.166488e+00
train#24960, train loss: 1.141231e+00
train#24970, train loss: 1.831663e+00
train#24980, train loss: 1.035265e+00
train#24990, train loss: 1.484603e+00
train#25000, train loss: 1.357866e+00
checkpoint saved 25000
train#25010, train loss: 1.893850e+00
train#25020, train loss: 1.717696e+00
train#25030, train loss: 1.433236e+00
train#25040, train loss: 1.381081e+00
train#25050, train loss: 1.800792e+00
train#25060, train loss: 1.759274e+00
train#25070, train loss: 1.792528e+00
train#25080, train loss: 9.509951e-01
train#25090, train loss: 1.590394e+00
train#25100, train loss: 1.772465e+00
checkpoint saved 25100
train#25110, train loss: 9.491183e-01
train#25120, train loss: 1.891874e+00
train#25130, train loss: 9.657034e-01
train#25140, train loss: 1.195298e+00
train#25150, train loss: 1.338768e+00
train#25160, train loss: 2.062872e+00
train#25170, train loss: 1.701849e+00
"""